In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import trange, tqdm

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
input_files = r'C:\Users\svo0175\Documents\Work\Svetlomet\Hella_06_05_2021\Hrabova_test_long_nove_svetlo\Hrabova_test_long_nove_svetlo_cut_{0}.{1}'
# input_files = r'C:\Users\Petr\Documents\svetlomety\data\Hrabova_test_long_nove_svetlo\Hrabova_test_long_nove_svetlo_cut_{0}.{1}'

thresholds_arr = [10, 50, 70]
thresholds_arr = [50]

In [ ]:
dfs = {}
for k in thresholds_arr:
    dfs[k] = pd.read_parquet(input_files.format(k, 'parquet.gzip'), engine='pyarrow')

In [ ]:
df = dfs[50]

In [ ]:
df.shape

In [ ]:
df.describe()

##### Jelikož je ve všech sloupcích podobná popisná statistika je pro agregaci hranice detekované přechodové hrany zvolen průměr mezi všemi sloupci obrazu.

In [ ]:
df_avg = pd.DataFrame()
df_avg['FrameTimestamp_us'] = df.FrameTimestamp_us
df_avg['avg_edge_occurence'] = df.iloc[:, :-1].mean(axis=1)
print(df_avg.shape)
df_avg.head()

In [ ]:
df_avg.describe()

#### Zobrazení signálu

In [ ]:
df_tmp = df_avg.iloc[:50000]
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

In [ ]:
df_tmp = df_avg.iloc[50000:150000]
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

In [ ]:
df_tmp = df_avg
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

#### Kontrola velkých změn mezi snímky - podezrele mista v kvalite video zaznamu

In [ ]:
(df_avg - df_avg.shift())[['FrameTimestamp_us', 'avg_edge_occurence']].describe(percentiles=[.01, .1, .25, .5, .75, .9, .99])

In [ ]:
df_avg['edge_shift'] = df_avg.avg_edge_occurence - df_avg.shift().avg_edge_occurence
px.line(x=df_avg.FrameTimestamp_us, y=df_avg.edge_shift)

Z předchozího grafu lze vidět, že přechody mezi snímky odpovídají velikosti rozkmitu. Obsahují pouze jednotlivé výkyvy (vzniklé například přeskočením snímku) a nelze nalézt region, který by obsahoval větší počet podezřelých přechodů.

#### Detekce periody

Pro výpočet byl použit postup počítání přechodů "0" signálu v našem případě použit průměr jako hranice přechodu rozkmitu.

U výpočtu průměrné hladiny rozkmitu bylo zvoleno okno 800 záznamů z videa. Tato hodnota byla vypočtena z předpokladu alespoň 10 cyklů pro určení aktuální frekvence a nejnižší předpokládáne frekvenci 5 Hz.

Jelikož se ukázalo, že u detekce přechodu přes průměr dochází k nepřesnostem, byl použit průměr 5 záznamů pro vyhlazení originálního signálu. ( V původní verzi bylo použito okno 10, ale docházelo k zploštění až k průměru při specifických frekvencích nebo k přetočení fáze. )

In [ ]:
df_avg['smoothed_avg_edge_occurence'] = df_avg.rolling(5, center=True).avg_edge_occurence.mean()
df_avg['rolling_avg_edge_occurence'] = df_avg.rolling(800, center=True).avg_edge_occurence.mean()

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
df_tmp = df_avg.iloc[:130000]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'smoothed_avg_edge_occurence', 'rolling_avg_edge_occurence'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')

In [ ]:
df_avg.loc[df_avg.avg_edge_occurence>=df_avg.rolling_avg_edge_occurence, 'polarity'] = 1
df_avg.loc[df_avg.avg_edge_occurence<df_avg.rolling_avg_edge_occurence, 'polarity'] = -1
df_avg['polarity_shift_abs'] = abs(df_avg.polarity-df_avg.polarity.shift())
df_avg['polarity_shift'] = df_avg.polarity-df_avg.polarity.shift()
df_avg.loc[df_avg.smoothed_avg_edge_occurence>=df_avg.rolling_avg_edge_occurence, 'polarity_smoothed'] = 1
df_avg.loc[df_avg.smoothed_avg_edge_occurence<df_avg.rolling_avg_edge_occurence, 'polarity_smoothed'] = -1
df_avg['polarity_smoothed_shift_abs'] = abs(df_avg.polarity_smoothed-df_avg.polarity_smoothed.shift())
df_avg['polarity_smoothed_shift'] = df_avg.polarity_smoothed-df_avg.polarity_smoothed.shift()

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'rolling_avg_edge_occurence', 'polarity', 'polarity_shift', 'polarity_smoothed', 'polarity_smoothed_shift'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')

In [ ]:
df_tmp = df_avg[(df_avg.FrameTimestamp_us >= 244925000) & (df_avg.FrameTimestamp_us <= 245200000)]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'rolling_avg_edge_occurence', 'polarity', 'polarity_shift', 'polarity_smoothed', 'polarity_smoothed_shift'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')

#### Výpočet periody

In [ ]:
df_avg['frames_timediff_us'] = df_avg.FrameTimestamp_us - df_avg.FrameTimestamp_us.shift()
df_avg['periody_start_mark'] = 0
df_avg.loc[df_avg.polarity_smoothed_shift == 2, 'periody_start_mark'] = 1
df_avg['rolling_sum_frames_timediff_us'] = df_avg.rolling(800, center=True).frames_timediff_us.sum()
df_avg['rolling_sum_periody_start_mark'] = df_avg.rolling(800, center=True).periody_start_mark.sum()
df_avg['frequency'] = 1 / (df_avg.rolling_sum_frames_timediff_us / df_avg.rolling_sum_periody_start_mark / 10**6)

In [ ]:
df_avg.iloc[1100:1150,:]

In [ ]:
df_avg.frequency.describe()

In [ ]:
df_tmp = df_avg.iloc[10000:150000].reset_index()
px.line(df_tmp, x='index', y='frequency')

In [ ]:
px.line(df_avg, x='FrameTimestamp_us', y='frequency')

##### Několik omezení, se kterým zatím pracujeme:

Při výpočtu průměru se počítá s centrováním plovoucího okna, což nepočítá s real time zpracováním.



#### Vyhlazení pozorované frekvence v sekvenci skrze median

In [ ]:
df_avg.head()

In [ ]:
df_avg_filter = df_avg.dropna().copy()

In [ ]:
df_avg_filter['frequency_rolling_mean'] = df_avg_filter.frequency.rolling(10000, center=True).quantile(0.5)

In [ ]:
df_avg_filter['FrameTimestamp_s'] = df_avg_filter.FrameTimestamp_us / 10**6

In [ ]:
px.line(df_avg_filter, x='FrameTimestamp_s', y='frequency_rolling_mean')

In [ ]:
df_tmp = df_avg_filter.iloc[:130000]
px.line(df_tmp, x='FrameTimestamp_s', y='frequency_rolling_mean')

In [ ]:
df_avg_filter.frequency_rolling_mean.describe()

#### Souhrnná statistika z vibro testu světlometu

Provést popisnou statistiku výkyvů pro:
- výkyv pixelů
- výkyv mm
 
- sekundově (line chart)
- minutově (boxplot)

- dle Hz (boxplot)

**TODO:**

- ?? Zvážit zarovnání na začátek vibro testu (např frekvence = 4Hz). Abychom pak měli lepší výsledky časových agregací.

- ?? Od čeho počítat +- posun? 800 rolling avg se výrazně v čase mění, navíc při cca 40Hz je patrná velká změna v klouzavém průměru. Pokud by se hodnoty porovnávaly vůči globálnímu průměru, tak dochází k nesymetrickým rozdílům + a - výkyvů. Nebo začít s ručně identifikovanou hodnotou z počátku záznamu před spuštění vibro testu.


In [ ]:
df_tmp = df_avg_filter.iloc[:130000]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_s'], value_vars=['avg_edge_occurence', 'rolling_avg_edge_occurence', 'frequency_rolling_mean'])
px.line(df_tmp, x='FrameTimestamp_s', y='value', color='variable')

In [ ]:
df_avg_filter.avg_edge_occurence.describe()

In [ ]:
# rucne urcena y_0 hladina z pocatku grafu
y_0 = 409.5
df_avg_filter['deviation_pixel'] = df_avg_filter.avg_edge_occurence - y_0
df_avg_filter['deviation_mm'] = df_avg_filter.deviation_pixel / 2
deviation_metric = 'deviation_mm'

In [ ]:
px.histogram(df_avg_filter, x=deviation_metric, nbins=100)

#### Klouzavý výpis pro sekundový přehled

Výhodou je kontinuální změna v průběhu testu. Nevýhodou je naivní způsob okno = 400 záznamů (i přesto, že to nemusí úplně sedět).

In [ ]:
df_400_rolling = df_avg_filter[deviation_metric].rolling(400).agg({'min': 'min', 'max': 'max', 'mean': 'mean'})
df_400_rolling['percentil_10'] = df_avg_filter[deviation_metric].rolling(400).quantile(0.1)
df_400_rolling['percentil_90'] = df_avg_filter[deviation_metric].rolling(400).quantile(0.9)
df_400_rolling['FrameTimestamp_s'] = df_avg_filter.FrameTimestamp_s

In [ ]:
df_tmp = df_400_rolling.iloc[:130000]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_s'], value_vars=['min', 'max', 'mean', 'percentil_10', 'percentil_90'])
fig = px.line(df_tmp, x='FrameTimestamp_s', y='value', color='variable', title=f'Klouzavá statistika rozptylů během vibračního testu (sloupec {deviation_metric})')
# fig.add_hline(y=0, line_width=1, line_dash='dash', line_color='black', opacity=0.7)
fig.show()

#### Minutová agregace vibračního testu

In [ ]:
t_0 = df_avg_filter.iloc[0, :].FrameTimestamp_s
df_avg_filter['elapsed_test_time_s'] = df_avg_filter.FrameTimestamp_s - t_0
df_avg_filter['elapsed_test_time_min'] = df_avg_filter.elapsed_test_time_s / 60
df_avg_filter['elapsed_test_time_min_floor'] =  np.floor(df_avg_filter.elapsed_test_time_min)

In [ ]:
df_avg_filter.elapsed_test_time_min_floor.describe()

In [ ]:
fig = px.box(df_avg_filter, x='elapsed_test_time_min_floor', y=deviation_metric)
# fig.add_hline(y=0, line_width=1, line_dash='dash', line_color='black', opacity=0.7)
fig.show()

In [ ]:
px.bar(df_avg_filter.groupby('elapsed_test_time_min_floor')[deviation_metric].mean().reset_index(),
       x='elapsed_test_time_min_floor', y=deviation_metric,
       title='Mean deviation in minutes of test')

#### Agregace výsledku testu pro frekvence vibračního zařízení

In [ ]:
# frekvence jsou cca od 3.8-61 Hz, vyhlazena frekvence je 6-58Hz
# tahle buňka by možná zasloužila refactor na vektorové přiřazení hodnoty
def get_frequency_bin_name(x):    
    value = x.frequency_rolling_mean
    if pd.isnull(value):
        return np.nan
    assert value > 0
    if value <= 10:
        return '(0; 10> Hz'
    elif value <= 15:
        return '(10; 15> Hz'
    elif value <= 20:
        return '(15; 20> Hz'
    elif value <= 25:
        return '(20; 25> Hz'
    elif value <= 30:
        return '(25; 30> Hz'
    elif value <= 35:
        return '(30; 35> Hz'
    elif value <= 40:
        return '(35; 40> Hz'
    elif value <= 45:
        return '(40; 45> Hz'
    elif value <= 50:
        return '(45; 50> Hz'
    elif value <= 55:
        return '(50; 55> Hz'
    elif value <= 65:
        return '(55; 65> Hz'
    else:
        return np.nan
    
df_avg_filter['frequency_bin'] = df_avg_filter[['frequency_rolling_mean']].apply(get_frequency_bin_name, axis=1)
df_avg_filter.dropna(inplace=True)

In [ ]:
df_avg_filter['frequency_bin'].describe()

In [ ]:
px.bar(df_avg_filter.groupby('frequency_bin')[deviation_metric].mean().reset_index(),
       x='frequency_bin', y=deviation_metric,
       title='Mean deviation for frequency bins of test')

In [ ]:
fig = px.box(df_avg_filter, x='frequency_bin', y=deviation_metric)
# fig.add_hline(y=0, line_width=1, line_dash='dash', line_color='black', opacity=0.7)
fig.show()

# K diszkuzi
- Šířka 256px je možná moc - může se jít do poloviny px
    - Je pak možné žvýšit i FPS
- Vzdálenost kamery, výška stativu a zešikmení - to by se mělo odstranit
- Vyšší rozptyl u vyšších frekvencí
    - Může být způsobeno konstrukčně u světlometu
    - Nepřesnost vibrační plošiny
    - Limit snímání obrazu - málo vzorků na vyšších frekvencích?
   